In [3]:
import glob
import hashlib
import json
import math
import os
import random
import copy
import shutil
import argparse
import time
from pathlib import Path
from itertools import repeat
import matplotlib.pyplot as plt
import cv2
import logging
import numpy as np
import scipy.sparse as ssp
import networkx as nx
import torch
import torch.nn.functional as F
from PIL import ExifTags, Image, ImageOps, ImageDraw, ImageFont
from torch.utils.data import DataLoader, Dataset, dataloader, distributed
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import transforms as T
from tool.utils import *
from data import *
from engine import train_one_epoch, evaluate
from metric import *
from tensorized_GNN import BatchedGNNlayer

# 
root_dir = '/nasdata2/khj/objectdetection/BMw'
IMG_FORMATS = ['bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp','gif'] 
img_dir, lab_dir = root_dir + '/images/', root_dir + '/labels/'


'''
Read caches
'''
f = []  # image files
for p in path if isinstance(img_dir, list) else [img_dir]:
    p = Path(p)
    f += glob.glob(str(p / '**' / '*.*'), recursive=True)
img_files = sorted(x.replace('/', os.sep) for x in f if x.split('.')[-1].lower() in IMG_FORMATS)
label_files = img2label_paths(img_files)  # labels

print('Cached images / labels : {}, and {}'.format(len(img_files),len(label_files)))

'''
Load data
'''
img_all, label_all = [], []
nins = 0
for sample in range(len(img_files)):
    # read
    img, labels = read_data(img_files,label_files,xywhn=False,subj_index=sample,img_size=608)
    if labels.ndim == 1: 
        labels = labels.reshape(1,-1)
    if np.shape(labels)[-1] == 0: 
        continue
    nins += np.shape(labels)[0]

    # stack
    img_all.append(img)
    label_all.append(labels)

Cached images / labels : 84, and 84


In [4]:
print(len(img_all))
print(np.shape(img_all[0]))

print(label_all[0])

84
(608, 608, 3)
[[25.        0.253285  0.470464  0.261314  0.447257]
 [25.        0.50146   0.512658  0.272993  0.316456]
 [25.        0.736496  0.56962   0.179562  0.253165]]


In [5]:
lab = label_all[0][:,1:]
bboxes = xywhn2xyxy(lab,608,608)

print(bboxes)

[[ 74.557824 150.075984 233.436736 422.00824 ]
 [221.897808 215.49344  387.877552 407.898688]
 [393.20272  269.3668   502.376416 423.29112 ]]
